## Velocity-Oriented Analytics (Deviation from True Streaming)

This notebook demonstrates velocity-oriented data processing through a controlled
micro-batch simulation rather than true real-time streaming. While Apache Spark
Structured Streaming was initially considered, platform-specific limitations led to
the adoption of a bounded micro-batch approach.

By processing the dataset incrementally in fixed-size batches and recomputing analytics
over time, this approach captures the core principles of data velocity, including
incremental computation, temporal ordering, and evolving insights, while ensuring
reproducibility and stable execution.


In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("ProstateCancerVelocitySimulation")
    .getOrCreate()
)

spark


In [ ]:
import os

data_path = os.path.join("..", "data", "prostate_cancer_prediction.csv")

df = spark.read.csv(data_path, header=True, inferSchema=True)
df.printSchema()


#### Adding synthetic time dimension


In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

df_with_time = df.withColumn(
    "arrival_id",
    monotonically_increasing_id()
)

df_with_time.show(5)


#### Define Micro-Batch Parameters

In [ ]:
BATCH_SIZE = 100
MAX_BATCHES = 5   # limits runtime intentionally

TOTAL_ROWS = df_with_time.count()

print(f"Total records: {TOTAL_ROWS}")
print(f"Micro-batch size: {BATCH_SIZE}")
print(f"Maximum batches to process: {MAX_BATCHES}")



#### Micro-Batch Processing Loop

In [ ]:
from pyspark.sql.functions import avg
import time

current_offset = 0
batch_number = 1

while current_offset < TOTAL_ROWS and batch_number <= MAX_BATCHES:
    print(f"\n=== Processing Micro-Batch {batch_number} ===")
    print(f"Time: {time.strftime('%H:%M:%S')}")
    
    micro_batch = (
        df_with_time
        .filter(
            (df_with_time.arrival_id >= current_offset) &
            (df_with_time.arrival_id < current_offset + BATCH_SIZE)
        )
    )
    
    result = micro_batch.groupBy("Cancer_Stage").agg(
        avg("PSA_Level").alias("avg_psa")
    )
    
    result.show()
    
    current_offset += BATCH_SIZE
    batch_number += 1
    
    # Simulate delay between arrivals
    time.sleep(2)

print("\nVelocity simulation completed successfully.")


## Summary

The velocity simulation demonstrates how analytical results can be updated as new data
arrives incrementally. By recomputing metrics over successive micro-batches, this
notebook illustrates the impact of data velocity on analytics without requiring
continuous streaming infrastructure.

This approach complements the batch analytics by showing how population-level insights
can be observed dynamically over time.
